In [1]:
def parse_input(file_path):
    """Read and parse the input file."""
    with open(file_path, "r") as file:
        lines = file.read().strip().split("\n")

    machines = []
    for i in range(0, len(lines), 3):
        button_a = tuple(map(int, lines[i].split(":")[1].strip().replace("X+", "").replace("Y+", "").split(",")))
        button_b = tuple(map(int, lines[i + 1].split(":")[1].strip().replace("X+", "").replace("Y+", "").split(",")))
        prize = tuple(map(int, lines[i + 2].split(":")[1].strip().replace("X=", "").replace("Y=", "").split(",")))
        machines.append({"A": button_a, "B": button_b, "prize": prize})

    return machines

def solve_diophantine(a, b, c, max_presses=None):
    """Solve the linear Diophantine equation a*n_A + b*n_B = c."""
    from math import gcd

    g = gcd(a, b)
    if c % g != 0:
        return []  # No solution exists

    # Simplify the equation
    a, b, c = a // g, b // g, c // g

    # Extended Euclidean Algorithm to find one solution
    def extended_gcd(x, y):
        if y == 0:
            return x, 1, 0
        g, x1, y1 = extended_gcd(y, x % y)
        return g, y1, x1 - (x // y) * y1

    _, x0, y0 = extended_gcd(a, b)
    x0 *= c
    y0 *= c

    # Generate all solutions within bounds
    solutions = []
    k = 0
    while True:
        n_A = x0 + k * b
        n_B = y0 - k * a
        if n_A < 0 or n_B < 0:
            if k > 0:
                break
            k += 1
            continue
        if max_presses is not None and (n_A > max_presses or n_B > max_presses):
            if k > 0:
                break
            k += 1
            continue
        solutions.append((n_A, n_B))
        k += 1

    return solutions

def find_min_cost(machine, max_presses=None):
    dx_A, dy_A = machine["A"]
    dx_B, dy_B = machine["B"]
    x_prize, y_prize = machine["prize"]

    # Solve for X and Y separately
    x_solutions = solve_diophantine(dx_A, dx_B, x_prize, max_presses)
    y_solutions = solve_diophantine(dy_A, dy_B, y_prize, max_presses)

    # Find common solutions
    min_cost = float("inf")
    for x_sol in x_solutions:
        for y_sol in y_solutions:
            if x_sol == y_sol:
                n_A, n_B = x_sol
                cost = 3 * n_A + n_B
                min_cost = min(min_cost, cost)

    return min_cost if min_cost != float("inf") else None

def solve_puzzle(file_path, prize_offset=0, max_presses=None):
    machines = parse_input(file_path)

    # Apply prize offset
    for machine in machines:
        machine["prize"] = (
            machine["prize"][0] + prize_offset,
            machine["prize"][1] + prize_offset
        )

    total_cost = 0
    prizes_won = 0

    for machine in machines:
        cost = find_min_cost(machine, max_presses)
        if cost is not None:
            prizes_won += 1
            total_cost += cost

    return prizes_won, total_cost

if __name__ == "__main__":
    input_file = "input.txt"  # Replace with your input file path

    # Part 1
    prizes_won, total_cost = solve_puzzle(input_file, prize_offset=0, max_presses=100)
    print(f"Part 1: Prizes won: {prizes_won}, Total cost: {total_cost}")

    # Part 2
    prizes_won, total_cost = solve_puzzle(input_file, prize_offset=10000000000000, max_presses=None)
    print(f"Part 2: Prizes won: {prizes_won}, Total cost: {total_cost}")


IndexError: list index out of range

In [2]:
import sys
import time
from typing import Optional

class PuzzleContext:
    data: str
    def __init__(self, year: int, day: int):
        self._year = year
        self._day = day
        self.data = None
        self._start_times = [None, None]
        self._end_times = [None, None]

    def __enter__(self):
        self.data = self._get_data()
        self._start_times[0] = time.time()
        return self

    def __exit__(self, type, value, traceback):
        duration = sum(t2-t1 for t1, t2 in zip(self._start_times, self._end_times) if t1 and t2)
        print(f"Total running time: {duration} sec")

    @property
    def groups(self):
        return self.data.split("\n\n")

    @property
    def lines(self):
        return self.data.split("\n")

    @property
    def nonempty_lines(self):
        return [l for l in self.lines if l]

    def _get_data(self):
        if self._is_running_on_sample():
            input_path = f"s{sys.argv[1]}.txt"
            with open(input_path, "r") as f:
                return f.read()
        else:
            from aocd import get_data
            return get_data(year=self._year, day=self._day)

    def submit(self, part: int, ans: Optional[str]):
        if part not in [1, 2]:
            raise ValueError(f"Part must be either 1 or 2, was {part}")
        if ans is None:
            print(f"Skipping submission for part {part}")
            return
        self._end_times[part-1] = time.time()
        self._log_answer(ans, part)
        if not self._is_running_on_sample():
            from aocd import submit
            submit(ans, part=["a", "b"][part-1], day=self._day, year=self._year)
        if part == 1:
            self._start_times[part] = time.time()
        
    def _is_running_on_sample(self):
        return len(sys.argv) > 1

    def _log_answer(self, ans: str, part: int):
        if self._start_times[part-1] is None:
            t_start = self._start_times[0]
        else:
            t_start = self._start_times[part-1]
        duration = self._end_times[part-1] - t_start
        
        print(f"Part {part}: {ans} ({duration} sec)")

In [3]:
from collections import *
from typing import *
from heapq import *
from dataclasses import dataclass
from puzzle import PuzzleContext
from utils import *
import itertools as itt
import functools as ft

def solve(xa: int, ya: int, xb: int, yb: int, x: int, y: int) -> tuple[int, int] | None:
    A = xa*y-x*ya
    B = xa*yb-xb*ya
    if A % B != 0:
        return None
    b = A // B
    C = x - b*xb
    D = xa
    if C % D != 0:
        return None
    a = C//D
    if a >= 0 and b >= 0:
        return (a, b)
    return None

with PuzzleContext(year=2024, day=13) as ctx:
    ans1, ans2 = 0, 0
    delta = 10000000000000
    for g in ctx.groups:
        xa, ya, xb, yb, x, y = ints(g)
        
        s = solve(xa, ya, xb, yb, x, y)
        if s is not None:
            a, b = s
            ans1 += 3*a+b
        
        s = solve(xa, ya, xb, yb, x+delta, y+delta)
        if s is not None:
            a, b = s
            ans2 += 3*a+b

    ctx.submit(1, str(ans1) if ans1 else None)
    ctx.submit(2, str(ans2) if ans2 else None)

ModuleNotFoundError: No module named 'puzzle'

In [4]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "OFFSET = 10000000000000  \n",
    "\n",
    "def parse_button_line(line):\n",
    "\n",
    "    line = line.strip()\n",
    "    _, coords = line.split(\":\", 1)\n",
    "    coords = coords.strip()\n",
    "    x_part, y_part = coords.split(\",\")\n",
    "    x_part = x_part.strip()  \n",
    "    y_part = y_part.strip() \n",
    "    Ax = int(x_part[1:])  \n",
    "    Ay = int(y_part[1:]) \n",
    "    return Ax, Ay\n",
    "\n",
    "def parse_prize_line(line):\n",
    "    line = line.strip()\n",
    "    _, coords = line.split(\":\", 1)\n",
    "    coords = coords.strip()\n",
    "    x_part, y_part = coords.split(\",\")\n",
    "    x_part = x_part.strip() \n",
    "    y_part = y_part.strip() \n",
    "    Px = int(x_part[2:])  \n",
    "    Py = int(y_part[2:])  \n",
    "    return Px, Py\n",
    "\n",
    "def min_cost_for_prize_bruteforce(Ax, Ay, Bx, By, Px, Py, max_presses=100):\n",
    "    # part 1 \n",
    "    min_cost = None\n",
    "    for nA in range(max_presses+1):\n",
    "        for nB in range(max_presses+1):\n",
    "            if Ax*nA + Bx*nB == Px and Ay*nA + By*nB == Py:\n",
    "                cost = 3*nA + nB\n",
    "                if min_cost is None or cost < min_cost:\n",
    "                    min_cost = cost\n",
    "    return min_cost\n",
    "\n",
    "def solve_machine_cramer(Ax, Ay, Bx, By, Px, Py):\n",
    "\n",
    "    ## part 2 cramer\n",
    "    Det = Ax*By - Ay*Bx\n",
    "    if Det == 0:\n",
    "        return None\n",
    "\n",
    "    nA_num = Px*By - Py*Bx\n",
    "    nB_num = Ax*Py - Ay*Px\n",
    "\n",
    "    if nA_num % Det != 0 or nB_num % Det != 0:\n",
    "        return None\n",
    "\n",
    "    nA = nA_num // Det\n",
    "    nB = nB_num // Det\n",
    "\n",
    "    if nA < 0 or nB < 0:\n",
    "        return None\n",
    "\n",
    "    cost = 3*nA + nB\n",
    "    return cost\n",
    "\n",
    "def main():\n",
    "\n",
    "    with open(\"input.txt\", \"r\") as f:\n",
    "        raw_lines = [line.strip() for line in f]\n",
    "    lines = [line for line in raw_lines if line]  \n",
    "\n",
    "    machines = []\n",
    "    for i in range(0, len(lines), 3):\n",
    "        if i+2 < len(lines):\n",
    "            A_line = lines[i]\n",
    "            B_line = lines[i+1]\n",
    "            P_line = lines[i+2]\n",
    "\n",
    "            Ax, Ay = parse_button_line(A_line)\n",
    "            Bx, By = parse_button_line(B_line)\n",
    "            Px, Py = parse_prize_line(P_line)\n",
    "\n",
    "            machines.append((Ax, Ay, Bx, By, Px, Py))\n",
    "\n",
    "    costs_part1 = []\n",
    "    for (Ax, Ay, Bx, By, Px, Py) in machines:\n",
    "        c = min_cost_for_prize_bruteforce(Ax, Ay, Bx, By, Px, Py, 100)\n",
    "        if c is not None:\n",
    "            costs_part1.append(c)\n",
    "    max_prizes_part1 = len(costs_part1)\n",
    "    total_cost_part1 = sum(costs_part1)\n",
    "\n",
    "    costs_part2 = []\n",
    "    for (Ax, Ay, Bx, By, Px, Py) in machines:\n",
    "        Px_off = Px + OFFSET\n",
    "        Py_off = Py + OFFSET\n",
    "        c = solve_machine_cramer(Ax, Ay, Bx, By, Px_off, Py_off)\n",
    "        if c is not None:\n",
    "            costs_part2.append(c)\n",
    "    max_prizes_part2 = len(costs_part2)\n",
    "    total_cost_part2 = sum(costs_part2)\n",
    "\n",
    "    print(\"Part 1 Results:\")\n",
    "    print(f\"The most prizes you can possibly win: {max_prizes_part1}\")\n",
    "    print(f\"The fewest tokens to spend to win all these prizes: {total_cost_part1}\")\n",
    "\n",
    "    print(\"\\nPart 2 Results (with 10 trillion offset):\")\n",
    "    print(f\"The most prizes you can possibly win: {max_prizes_part2}\")\n",
    "    print(f\"The fewest tokens you would have to spend to win all these prizes: {total_cost_part2}\")\n",
    "\n",
    "main()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.1"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}

NameError: name 'null' is not defined

In [1]:
data = []
with open("input2.txt", 'r') as f:
    data = f.read().strip('\n').split('\n')

offset = 10000000000000
groups = '\n'.join(data).split('\n\n')
total = 0
for group in groups:
    a_str, b_str, p_str = group.split('\n')

    ax, ay = [int(x[2:]) for x in a_str[10:].split(', ')]
    bx, by = [int(x[2:]) for x in b_str[10:].split(', ')]
    px, py = [int(x[2:]) for x in p_str[7:].split(', ')]

    px += offset
    py += offset

    m = (px * by - py * bx) // (ax * by - ay * bx)
    if m * (ax * by - ay * bx) != (px * by - py * bx):
        continue
    n = (py - ay * m) // by
    if n * by != (py - ay * m):
        continue

    total += 3 * m + n

print(total)

ValueError: invalid literal for int() with base 10: ': X+27'